In [10]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
import nltk
from nltk.corpus import stopwords
import re
import string
nltk.download('stopwords')
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amk00\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
data = pd.read_csv("train.csv")

In [12]:
data.head()

,Sentence_id,New_Sentence,Type
0,GERRES15609,<html> b'Author and/or Review architecture/des...,Responsibility
1,PHERES15784,<html> b'Should be able to develop custom dyna...,Responsibility
2,GERREQ10457,<html> b'Experience in working cross\\u2010fun...,Requirement
3,GERSKL27235,"<html> b'Previous business experience, includi...",Skill
4,HONSSK18415,b'Delivering fast and right the first \\U0001f...,SoftSkill


In [72]:
data['sentence'] = data['New_Sentence'].str.replace(r'<[^<>]*>', '', regex=True)

In [73]:
stop_words = set(stopwords.words('english'))

In [13]:
data.head(5)

,Sentence_id,New_Sentence,Type
0,GERRES15609,<html> b'Author and/or Review architecture/des...,Responsibility
1,PHERES15784,<html> b'Should be able to develop custom dyna...,Responsibility
2,GERREQ10457,<html> b'Experience in working cross\\u2010fun...,Requirement
3,GERSKL27235,"<html> b'Previous business experience, includi...",Skill
4,HONSSK18415,b'Delivering fast and right the first \\U0001f...,SoftSkill


In [74]:
data.Type.unique()

array(['Responsibility', 'Requirement', 'Skill', 'SoftSkill', 'Education',
       'Experience'], dtype=object)

In [75]:
df= data
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df['labels']= label_encoder.fit_transform(df['Type'])
sentences = df.sentence.values


# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = [ sentence   for sentence in sentences]
labels = df.labels.values

In [79]:
STOPWORDS = set(stopwords.words('english'))
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [80]:
df['sentence'] = df['sentence'].apply(lambda text: cleaning_stopwords(text))

In [83]:
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
df['sentence']= df['sentence'].apply(lambda x: cleaning_punctuations(x))

In [85]:
xtrain, xtest, ytrain, ytest = train_test_split(df['sentence'], df['labels'], 
                                                            random_state=2018, test_size=0.1)


In [88]:
max_len = 70
tok = Tokenizer(num_words=2000)
tok.fit_on_texts(df['sentence'])
sequences = tok.texts_to_sequences(df['sentence'])
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
word_index = tok.word_index


In [94]:
len(df['labels'].unique())

6

In [108]:
Y=df['labels']
xtrain, xtest, ytrain, ytest = train_test_split(sequences_matrix, Y, 
                                                            random_state=2018, test_size=0.1)


In [176]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     200,
                     input_length=max_len))
model.add(SimpleRNN(200))
model.add(Dense(32, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(6, activation="softmax"))
model.compile(loss =SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 70, 200)           6081600   
                                                                 
 simple_rnn_10 (SimpleRNN)   (None, 200)               80200     
                                                                 
 dense_7 (Dense)             (None, 32)                6432      
                                                                 
 dense_8 (Dense)             (None, 64)                2112      
                                                                 
 dense_9 (Dense)             (None, 6)                 390       
                                                                 
Total params: 6,170,734
Trainable params: 6,170,734
Non-trainable params: 0
_________________________________________________________________


In [177]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                          min_delta = 0,
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True),
    tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5',monitor='val_accuracy',
                             mode='max',
                             save_best_only=True,
                             verbose=1),
]

In [179]:
model.fit(xtrain, ytrain, epochs=6, validation_data=(xtest, ytest), callbacks=my_callbacks) 

Epoch 1/6
1691/1691 [==============================] - ETA: 0s - loss: 0.9339 - accuracy: 0.6492
Epoch 1: val_accuracy improved from 0.69528 to 0.70725, saving model to best_model.h5
1691/1691 [==============================] - 135s 80ms/step - loss: 0.9339 - accuracy: 0.6492 - val_loss: 0.8018 - val_accuracy: 0.7073
Epoch 2/6
1691/1691 [==============================] - ETA: 0s - loss: 0.7693 - accuracy: 0.7143
Epoch 2: val_accuracy improved from 0.70725 to 0.72572, saving model to best_model.h5
1691/1691 [==============================] - 137s 81ms/step - loss: 0.7693 - accuracy: 0.7143 - val_loss: 0.7551 - val_accuracy: 0.7257
Epoch 3/6
1691/1691 [==============================] - ETA: 0s - loss: 0.7689 - accuracy: 0.7145
Epoch 3: val_accuracy did not improve from 0.72572
1691/1691 [==============================] - 128s 76ms/step - loss: 0.7689 - accuracy: 0.7145 - val_loss: 0.7387 - val_accuracy: 0.7237
Epoch 4/6
1691/1691 [==============================] - ETA: 0s - loss: 0.6998 

In [180]:
test = pd.read_csv("test.csv")
test['sentence'] = test['New_Sentence'].str.replace(r'<[^<>]*>', '', regex=True)
test['sentence'] = test['sentence'].apply(lambda text: cleaning_stopwords(text))
test['sentence']= test['sentence'].apply(lambda x: cleaning_punctuations(x))
test_sequences = tok.texts_to_sequences(test['sentence'])
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
test_scores = model.predict(test_sequences_matrix)
tscore=[]
for s in test_scores:
    tscore+=[int(np.where(s == s.max())[0])]
pred= label_encoder.inverse_transform(tscore)
csv =pd.DataFrame({'Sentence_id':test['Sentence_id'],'Type':pred})
csv.to_csv('submission.csv',index=False)


470/470 [==============================] - 4s 9ms/step
